In [1]:
import os
import time
import pandas as pd

from Bio import Entrez, SeqIO

from fasta_processing import dict_align_to_fasta, read_single_fasta, plain_to_fasta
from parse_psi_blast_results import parse_psiblast_xml, filter_psiblast_hits, update_df, save_genes, save_cdss_exons

Entrez.email = "artemvaskaa@gmail.com"

In [2]:
name_of_blast_res = "../Blast_res/schistosoma_haematobium_psi_blast.xml" # XML2 !!!
df = parse_psiblast_xml(name_of_blast_res)

In [3]:
df = filter_psiblast_hits(df, min_qc=0.8, min_ident=0.8, min_sbjct_len=500)

In [4]:
df = update_df(df)

In [6]:
save_genes(df)

In [7]:
save_cdss_exons(df)

---